# Purly Local Givens Rotations

In [55]:
import qiskit.quantum_info as qi
import scipy as sp
import numpy as np
import pandas as pd
import math
import scipy.linalg as ln

def I(N):
    label = ['I' for i in range(N)]
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def X(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'X'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Y(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'Y'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Z(i,N):
    label = ['I' for i in range(N)]
    label[N-1-i] = 'Z'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def c(i,N):
    label_1 = ['Z' for j in range(N-i-1)]
    label_2 = ['I' for j in range(N-i,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x+1j*y)

def cd(i,N):
    label_1 = ['Z' for j in range(N-i-1)]
    label_2 = ['I' for j in range(N-i,N)]
    label_x = label_1 + ['X'] + label_2
    label_y = label_1 + ['Y'] + label_2
    label_x = ''.join(label_x)
    label_y = ''.join(label_y)
    x = qi.Operator.from_label(label_x).data
    y = qi.Operator.from_label(label_y).data
    return 1/2*(x-1j*y)

def n(i,N):
    return Mdot([cd(i,N),c(i,N)])

def ad(n,N):
    out = cd(0,N)
    for i in range(1,N):
        out = out + np.exp(1j*2*np.pi*n*i/N)*cd(i,N)
    return 1/np.sqrt(N)*out

def a(n,N):
    out = c(0,N)
    for i in range(1,N):
        out = out + np.exp(-1j*2*np.pi*n*i/N)*c(i,N)
    return 1/np.sqrt(N)*out

import numpy as np


def Mdot(Ol):
    L = len(Ol)
    out = Ol[L-1]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[L-1-i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])


In [56]:
def f(i,n,N):
    return np.sqrt(1/(N))*np.exp(1j*2*np.pi*(i)*(n)/N)

def F(N): 
    return np.array([[f(i,n,N) for i in range(N)] for n in range(N)])

def psi0(N):
    y = [0 for i in range(2**N)]
    y[0] = 1
    return y

In [3]:
def ry(i,j,phi,N):
    M = (1+0*1j)*np.identity(N)
    M[i,i] = np.cos(phi)
    M[j,j] = np.cos(phi)
    M[i,j] = np.sin(phi)
    M[j,i] = -np.sin(phi)
    return M


def rz(j,phi,N):
    M = (1+0*1j)*np.identity(N)
    M[j,j] = np.exp(1j*phi)
    return M

def givens(i,j,F):
    phiz = 1j*np.log( F[j,i]/F[i,i] * np.abs(F[i,i])/np.abs(F[j,i]) )
    Fz =  Mdot([rz(j,phiz,4) , F])
    phi = np.arctan(Fz[j,i]/Fz[i,i])
    F_new = Mdot([ry(i,j,phi,4) , Fz])
    return F_new,phiz,phi

from qiskit import QuantumCircuit

def fswap(i,j,qc):
    qc.swap(i,j)
    qc.ry(np.pi/2,j)
    qc.cx(i,j)
    qc.ry(-np.pi/2,j)
    return qc
    

def R_cc(i,j,phi,qc):
    for l in range(i+1,j):
        qc = fswap(l-1,l,qc)
    qc.ry(-np.pi/2,j-1)
    qc.cx(j-1,j)
    qc.ry(-phi,j)
    qc.cx(j-1,j)
    qc.ry(np.pi/2,j-1)
    qc.ry(-np.pi/2,j)
    qc.cx(j,j-1)
    qc.ry(phi,j-1)
    qc.cx(j,j-1)
    qc.ry(np.pi/2,j)
    for l in range(j-1,i+1-1,-1):
        qc = fswap(l-1,l,qc)
    return qc

def G_cc(i,j,phi,phiz,qc):
    qc = R_cc(i,j,-phi,qc)
    qc.rz(-phiz,j)
    return qc


Let us redefine the givens rotation so that we can elinate an element specific collumn $c$ for any two row choices $ri$ and $rj$.

In [33]:
def givens(ri,rj,c,F):
    phiz = 1j*np.log( F[rj,c]/F[ri,c] * np.abs(F[ri,c])/np.abs(F[rj,c]) )
    Fz =  Mdot([rz(rj,phiz,4) , F])
    phi = np.arctan(Fz[rj,c]/Fz[ri,c])
    F_new = Mdot([ry(ri,rj,phi,4) , Fz])
    return F_new,phiz,phi



In [47]:
F0 = F(4)
F1,pz1,p1 = givens(2,3,0,F0)
F2,pz2,p2 = givens(1,2,0,F1)
F3,pz3,p3 = givens(0,1,0,F2)
F4,pz4,p4 = givens(2,3,1,F3)
F5,pz5,p5 = givens(1,2,1,F4)
F6,pz6,p6 = givens(2,3,2,F5)

In [48]:
pd.DataFrame(F6)

,0,1,2,3
0,1.000000e+00+0.000000e+00j,8.447994e-17+1.284693e-16j,8.447994e-17+6.123234e-17j,1.806281e-16-1.071235e-17j
1,5.084879e-17-6.451755e-17j,-1.000000e+00+1.108101e-16j,-1.160597e-16+5.190271e-17j,2.012130e-16-1.420463e-16j
2,-6.483586e-17-1.293403e-33j,-4.901010e-18+4.991470e-17j,1.000000e+00+3.017270e-17j,1.211688e-16-1.969183e-16j
3,5.084879e-17+6.451755e-17j,-3.535109e-17+4.991470e-17j,1.008758e-16+6.162976e-33j,-1.000000e+00-8.422827e-18j


In [61]:
from qiskit import QuantumCircuit, transpile, QuantumRegister,ClassicalRegister, execute

def slatter_circ(F0):
    F1,pz1,p1 = givens(2,3,0,F0)
    F2,pz2,p2 = givens(1,2,0,F1)
    F3,pz3,p3 = givens(0,1,0,F2)
    F4,pz4,p4 = givens(2,3,1,F3)
    F5,pz5,p5 = givens(1,2,1,F4)
    F6,pz6,p6 = givens(2,3,2,F5)
    ph0 = -1j*np.log(F6[0,0])
    ph1 = -1j*np.log(F6[1,1])
    ph2 = -1j*np.log(F6[2,2])
    ph3 = -1j*np.log(F6[3,3])
    qr = QuantumRegister(4)
    cr = ClassicalRegister(4)
    qc = QuantumCircuit(qr , cr)
    qc.x(2)
    qc.x(3)
    qc.rz(np.real(ph0),0)
    qc.rz(np.real(ph1),1)
    qc.rz(np.real(ph2),2)
    qc.rz(np.real(ph3),3)
    qc = G_cc(2,3,np.real(p6),np.real(pz6),qc)
    qc = G_cc(1,2,np.real(p5),np.real(pz5),qc)
    qc = G_cc(2,3,np.real(p4),np.real(pz4),qc)
    qc = G_cc(0,1,np.real(p3),np.real(pz3),qc)
    qc = G_cc(1,2,np.real(p2),np.real(pz2),qc)
    qc = G_cc(2,3,np.real(p1),np.real(pz1),qc)
    return qc

In [62]:
slatter_circ(F(4)).draw()

┌───────┐┌──────────┐                                                  »
q10_0: ┤ Rz(0) ├┤ Ry(-π/2) ├──────────────────────────────────────────────────»
       ├───────┤├──────────┤                                                  »
q10_1: ┤ Rz(π) ├┤ Ry(-π/2) ├──────────────────────────────────────────────────»
       └─┬───┬─┘└┬───────┬─┘┌──────────┐                     ┌─────────┐ ┌───┐»
q10_2: ──┤ X ├───┤ Rz(0) ├──┤ Ry(-π/2) ├──■───────────────■──┤ Ry(π/2) ├─┤ X ├»
         ├───┤   ├───────┴┐ └──────────┘┌─┴─┐┌─────────┐┌─┴─┐├─────────┴┐└─┬─┘»
q10_3: ──┤ X ├───┤ Rz(-π) ├─────────────┤ X ├┤ Ry(π/4) ├┤ X ├┤ Ry(-π/2) ├──■──»
         └───┘   └────────┘             └───┘└─────────┘└───┘└──────────┘     »
c10: 4/═══════════════════════════════════════════════════════════════════════»
                                                                              »
«                                                                        »
«q10_0: ─────────────────────────────────────────────────────────────────»
«                                                       ┌─────────┐ ┌───┐»
«q10_1: ──────────────────────■──────────────────────■──┤ Ry(π/2) ├─┤ X ├»
«       ┌──────────┐┌───┐   ┌─┴─┐   ┌─────────────┐┌─┴─┐├─────────┴┐└─┬─┘»
«q10_2: ┤ Ry(-π/4) ├┤ X ├───┤ X ├───┤ Ry(0.95532) ├┤ X ├┤ Ry(-π/2) ├──■──»
«       └──────────┘└─┬─┘┌──┴───┴──┐└─┬──────────┬┘└───┘└──────────┘     »
«q10_3: ──────────────■──┤ Ry(π/2) ├──┤ Rz(3π/4) ├───────────────────────»
«                        └─────────┘  └──────────┘                       »
«c10: 4/═════════════════════════════════════════════════════════════════»
«                                                                        »
«                                                                ┌─────────┐ »
«q10_0: ──────────────────────────■───────────────────────■──────┤ Ry(π/2) ├─»
«       ┌──────────────┐┌───┐   ┌─┴─┐    ┌─────────┐    ┌─┴─┐    ├─────────┴┐»
«q10_1: ┤ Ry(-0.95532) ├┤ X ├───┤ X ├────┤ Ry(π/3) ├────┤ X ├────┤ Ry(-π/2) ├»
«       └──────────────┘└─┬─┘┌──┴───┴──┐┌┴─────────┴┐┌──┴───┴───┐└──────────┘»
«q10_2: ──────────────────■──┤ Ry(π/2) ├┤ Rz(1.249) ├┤ Ry(-π/2) ├─────■──────»
«                            └─────────┘└───────────┘└──────────┘   ┌─┴─┐    »
«q10_3: ────────────────────────────────────────────────────────────┤ X ├────»
«                                                                   └───┘    »
«c10: 4/═════════════════════════════════════════════════════════════════════»
«                                                                            »
«            ┌───┐     ┌──────────┐   ┌───┐                               »
«q10_0: ─────┤ X ├─────┤ Ry(-π/3) ├───┤ X ├───────────────────────────────»
«            └─┬─┘     └──────────┘   └─┬─┘    ┌─────────┐   ┌───────┐    »
«q10_1: ───────■────────────────────────■──────┤ Ry(π/2) ├───┤ Rz(0) ├────»
«                                  ┌─────────┐ └──┬───┬──┘┌──┴───────┴───┐»
«q10_2: ────────────────────■──────┤ Ry(π/2) ├────┤ X ├───┤ Ry(-0.65906) ├»
«       ┌─────────────┐   ┌─┴─┐    ├─────────┴┐   └─┬─┘   └──────────────┘»
«q10_3: ┤ Ry(0.65906) ├───┤ X ├────┤ Ry(-π/2) ├─────■─────────────────────»
«       └─────────────┘   └───┘    └──────────┘                           »
«c10: 4/══════════════════════════════════════════════════════════════════»
«                                                                         »
«                                                                   »
«q10_0: ────────────────────────────────────────────────────────────»
«       ┌──────────┐                               ┌─────────┐ ┌───┐»
«q10_1: ┤ Ry(-π/2) ├─────■──────────────────────■──┤ Ry(π/2) ├─┤ X ├»
«       └──┬───┬───┘   ┌─┴─┐   ┌─────────────┐┌─┴─┐├─────────┴┐└─┬─┘»
«q10_2: ───┤ X ├───────┤ X ├───┤ Ry(0.95532) ├┤ X ├┤ Ry(-π/2) ├──■──»
«          └─┬─┘    ┌──┴───┴──┐└┬────────────┤└───┘└──────────┘     »
«q10_3: ─────■──────┤ Ry(π/2) ├─┤ Rz(1.1071) ├──────────────────────»
«                   └─────────┘ └────

In [63]:
#Test
F1,pz1,p1 = givens(2,3,0,F0)
F2,pz2,p2 = givens(1,2,0,F1)
F3,pz3,p3 = givens(0,1,0,F2)
F4,pz4,p4 = givens(2,3,1,F3)
F5,pz5,p5 = givens(1,2,1,F4)
F6,pz6,p6 = givens(2,3,2,F5)
ph0 = -1j*np.log(F6[0,0])
ph1 = -1j*np.log(F6[1,1])
ph2 = -1j*np.log(F6[2,2])
ph3 = -1j*np.log(F6[3,3])

qc = slatter_circ(F(4))
psi_tst = qi.Statevector.from_instruction(qc).data
phase = np.exp(-1j*ph0/2)*np.exp(-1j*ph1/2)*np.exp(-1j*ph2/2)*np.exp(-1j*ph3/2)
phase = phase*np.exp(1j*pz6/2)*np.exp(1j*pz5/2)*np.exp(1j*pz4/2)*np.exp(1j*pz3/2)*np.exp(1j*pz2/2)*np.exp(1j*pz1/2)
psi1 = Mdot([ad(3,4),ad(2,4),psi0(4)])
np.amax(np.abs( psi_tst - phase*psi1 ))

2.348854246544412e-16